# Constrained K-Means demo - Chicago Weather Dataset

## H2O K-Means algorithm

K-Means falls in the general category of clustering algorithms. Clustering is a form of unsupervised learning that tries to find structures in the data without using any labels or target values. Clustering partitions a set of observations into separate groupings such that observation in a given group is more similar to another observation in the same group than to another observation in a different group.

![kmeans](https://media0.giphy.com/media/12vVAGkaqHUqCQ/giphy.gif?cid=790b7611178aaedddb5b58de2ef94d55dc6c3feecd2d02f2&rid=giphy.gif)

More about H2O K-means Clustering: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/k-means.html

## Constrained K-Means algorithm in H2O

Using the `cluster_size_constraints` parameter, a user can set the minimum size of each cluster during the training by an array of numbers. The size of the array must be equal as the `k` parameter.

To satisfy the custom minimal cluster size, the calculation of clusters is converted to the Minimal Cost Flow problem. Instead of using the Lloyd iteration algorithm, a graph is constructed based on the distances and constraints. The goal is to go iteratively through the input edges and create an optimal spanning tree that satisfies the constraints.

![mcf](https://adared.ch/wp-content/uploads/2015/11/mcf.png)

More information about how to convert the standard K-means algorithm to the Minimal Cost Flow problem is described in this paper: https://pdfs.semanticscholar.org/ecad/eb93378d7911c2f7b9bd83a8af55d7fa9e06.pdf.

**Minimum-cost flow problem can be efficiently solved in polynomial time. Currently, the performance of this implementation of Constrained K-means algorithm is slow due to many repeatable calculations which cannot be parallelized and more optimized at H2O backend.**

Expected time with various sized data:
* 5 000 rows, 5 features   ~ 0h  4m  3s
* 10 000 rows, 5 features  ~ 0h  9m 21s
* 15 000 rows, 5 features  ~ 0h 22m 25s
* 20 000 rows, 5 features  ~ 0h 39m 27s
* 25 000 rows, 5 features  ~ 1h 06m  8s
* 30 000 rows, 5 features  ~ 1h 26m 43s
* 35 000 rows, 5 features  ~ 1h 44m  7s
* 40 000 rows, 5 features  ~ 2h 13m 31s
* 45 000 rows, 5 features  ~ 2h  4m 29s
* 50 000 rows, 5 features  ~ 4h  4m 18s

(OS debian 10.0 (x86-64), processor Intel© Core™ i7-7700HQ CPU @ 2.80GHz × 4, RAM 23.1 GiB)

## Shorter time using Aggregator Model

To solve Constrained K-means in a shorter time, you can used the H2O Aggregator model to aggregate data to smaller size first and then pass these data to the Constrained K-means model to calculate the final centroids to be used with scoring. The results won't be as accurate as a result from a model with the whole dataset. However, it should help solve the problem of a huge datasets.

However, there are some assumptions:
* the large dataset has to consist of many similar data points - if not, the insensitive aggregation can break the structure of the dataset
* the resulting clustering may not meet the initial constraints exactly when scoring (this also applies to Constrained K-means model, scoring use only result centroids to score and no constraints defined before)

The H2O Aggregator method is a clustering-based method for reducing a numerical/categorical dataset into a dataset with fewer rows. Aggregator maintains outliers as outliers but lumps together dense clusters into exemplars with an attached count column showing the member points.

More about H2O Aggregator: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/aggregator.html

In [1]:
import sys
sys.path.append("/home/mori/Documents/h2o/env/h2o-env/lib/python3.7/site-packages")
sys.path.append("/home/mori/Documents/h2o/code/h2o-3/h2o-py/build")

In [2]:
# run h2o Kmeans

# Import h2o library
import h2o
from h2o.estimators import H2OKMeansEstimator

# init h2o cluster
h2o.init(strict_version_check=False, url="http://10.30.0.22:54321")

versionFromGradle='3.29.0',projectVersion='3.29.0.99999',branch='maurever_PUBDEV-6447_constrained_kmeans_improvement',lastCommitHash='162ceb18eae8b773028f27b284129c3ef752d001',gitDescribe='jenkins-master-4952-11-g162ceb18ea-dirty',compiledOn='2020-02-20 15:01:59',compiledBy='mori'
Checking whether there is an H2O instance running at http://10.30.0.22:54321 . connected.


H2O cluster uptime:,19 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.29.0.99999
H2O cluster version age:,9 hours and 56 minutes
H2O cluster name:,mori
H2O cluster total nodes:,1
H2O cluster free memory:,5.109 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


## Data - Chicago Weather dataset

- 5162 rows
- 5 features (monht, day, year, maximal temperature, mean teperature)

In [3]:
# load data
import pandas as pd

data = pd.read_csv("../../smalldata/chicago/chicagoAllWeather.csv")
data = data.iloc[:,[1, 2, 3, 4, 5]]
print(data.shape)
data.head()

(5162, 5)


,month,day,year,maxTemp,meanTemp
0,1,1,2001,23.0,14.0
1,1,2,2001,18.0,12.0
2,1,3,2001,28.0,18.0
3,1,4,2001,30.0,24.0
4,1,5,2001,36.0,30.0


In [4]:
# import time to measure elapsed time
from timeit import default_timer as timer
from datetime import timedelta
import time

start = timer()
end = timer()
print("Time:", timedelta(seconds=end-start))

Time: 0:00:00.000043


Expected time with various sized data:
* 5 000 rows, 5 features   ~ 0h  0m 35s
* 10 000 rows, 5 features  ~ 0h  1m 54s
* 15 000 rows, 5 features  ~ 0h  4m 19s
* 20 000 rows, 5 features  ~ 0h  7m 46s
* 25 000 rows, 5 features  ~ 0h 11m 44s
* 30 000 rows, 5 features  ~ 0h 17m 14s
* 35 000 rows, 5 features  ~ 0h 22m 53s
* 40 000 rows, 5 features  ~ h m s
* 45 000 rows, 5 features  ~ h m s
* 50 000 rows, 5 features  ~ h m s

## Traditional K-means

## Constrained K-means

In [5]:
data_h2o = h2o.H2OFrame(data)

# run h2o Kmeans to get good starting points
h2o_km = H2OKMeansEstimator(k=3, init="furthest", standardize=True)
start = timer()
h2o_km.train(training_frame=data_h2o)
end = timer()

user_points = h2o.H2OFrame(h2o_km.centers())

# show details
h2o_km.show()
time_km = timedelta(seconds=end-start)
print("Time:", time_km)
h2o.remove(data_h2o)

Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1582238555392_1


Model Summary: 


,,number_of_rows,number_of_clusters,number_of_categorical_columns,number_of_iterations,within_cluster_sum_of_squares,total_sum_of_squares,between_cluster_sum_of_squares
0,,5162.0,3.0,0.0,10.0,13948.754926,25779.0,11830.245074




ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 13948.749743334341
Total Sum of Square Error to Grand Mean: 25778.999972296842
Between Cluster Sum of Square Error: 11830.2502289625

Centroid Statistics: 


,,centroid,size,within_cluster_sum_of_squares
0,,1.0,1134.0,3047.610882
1,,2.0,1527.0,4341.602251
2,,3.0,2501.0,6559.536610



Scoring History: 


,,timestamp,duration,iterations,number_of_reassigned_observations,within_cluster_sum_of_squares
0,,2020-02-20 23:43:02,0.041 sec,0.0,NaN,NaN
1,,2020-02-20 23:43:02,0.119 sec,1.0,5162.0,21533.089233
2,,2020-02-20 23:43:02,0.141 sec,2.0,475.0,16141.173442
3,,2020-02-20 23:43:02,0.147 sec,3.0,369.0,15747.276295
4,,2020-02-20 23:43:02,0.154 sec,4.0,389.0,15214.763875
5,,2020-02-20 23:43:02,0.160 sec,5.0,321.0,14566.183474
6,,2020-02-20 23:43:02,0.167 sec,6.0,236.0,14126.147235
7,,2020-02-20 23:43:02,0.173 sec,7.0,69.0,13960.646170
8,,2020-02-20 23:43:02,0.178 sec,8.0,25.0,13950.107800
9,,2020-02-20 23:43:02,0.183 sec,9.0,10.0,13948.870202


Time: 0:00:00.338255


In [6]:
data_h2o = h2o.H2OFrame(data)
# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=[1000, 2000, 1000], standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
Time: 0:01:35.819023


In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data]))
i = 2
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data]))
i = 3
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data]))
i = 4
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data, data]))
i = 5
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data, data, data]))
i = 6
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data, data, data, data]))
i = 7
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

In [7]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data, data, data, data, data]))
i = 8
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
Time: 1:17:05.203777


In [8]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data, data, data, data, data, data]))
i = 9
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
Time: 1:38:15.584080


In [ ]:
data_h2o = h2o.H2OFrame(pd.concat([data, data, data, data, data, data, data, data, data, data]))
i = 10
constraints = [i*1000, i*2000, i*1000]

# run h2o constrained Kmeans
h2o_km_co = H2OKMeansEstimator(k=3, user_points=user_points, cluster_size_constraints=constraints, standardize=True)
start = timer()
h2o_km_co.train(training_frame=data_h2o)
end = timer()

# show details
# h2o_km_co.show()
time_km_co = timedelta(seconds=end-start)
print("Time:", time_km_co)
h2o.remove(data_h2o)

Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |███████████████